[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_1%5B5%5D.pdf)

In [1]:
from common import *

In [2]:
class IncompatibleMatricesException(Exception):
    def __init__(self, m1, m2):
        message = "Not true that {0}, {1} is compatible with {2}, {3}." \
            .format(m1.height, m1.width, m2.height, m2.width)
        super(IncompatibleMatricesException, self).__init__(message)
        
class InvalidDimensionException(Exception):
    def __init__(self, m):
        message = "Invalid dimensions {0}, {1} from given matrix." \
            .format(m.height, m.width)
        super(InvalidDimensionException, self).__init__(message)

In [3]:
def matrix_multiply(fst, snd):
    zip_snd = list(zip(*snd))
    
    return [[sum(a * b for a, b in zip(row_fst, col_snd)) 
                 for col_snd in zip_snd] 
                    for row_fst in fst]

def scalar(scalar, elements, op):
    return [[op(e, scalar) for e in row] for row in elements]

def elementwise(fst, snd, operation):
    o = lambda z: [operation(a, b) for a, b in zip(z[0], z[1])]
    return [o(zipped) for zipped in zip(fst, snd)]

In [45]:
import numbers, math, itertools, copy

class Matrix:
    
    def __init__(self, elements_or_matrix):
        if isinstance(elements_or_matrix, Matrix):
            elements_or_matrix = elements_or_matrix.elements
        if type(elements_or_matrix) != list:
            raise Exception("Expecting list as argument")
        if type(elements_or_matrix[0]) != list:
            # wrap so we always have 2d matrix.
            elements_or_matrix = [elements_or_matrix]
            
            
        self.elements = copy.deepcopy(elements_or_matrix)
        
    @property
    def width(self):
        return 0 if self.height == 0 \
                 else len(self.elements) \
                    if type(self.elements[0]) != list \
                    else len(self.elements[0])
    
    @property
    def height(self):
        return len(self.elements)
    
    @property
    def T(self):
        return Matrix([list(col) for col in zip(*self.elements)])
    
    def __getitem__(self, key):
        if len(self.elements[0]) == 1:
            return self.elements[key][0]
        return self.elements.__getitem__(key)
    
    def __setitem__(self, key, item):
        if len(self.elements[0]) == 1:
            self.elements[key][0] = item
        else:
            self.elements.__setitem__(key, item)
    
    def __add__(self, other):
        return Matrix(self.elements).__iadd__(other)
        
    def __sub__(self, other):
        return Matrix(self.elements).__isub__(other)
    
    def __iadd__(self, rhs):
        if self.height != rhs.height \
            or self.width != rhs.width:
            raise IncompatibleMatricesException(self, rhs)
        
        self.elements = elementwise(
            self.elements, rhs.elements, lambda a, b: a + b)
        
        return self
    
    def __isub__(self, rhs):
        if self.height != rhs.height \
            or self.width != rhs.width:
            raise IncompatibleMatricesException(self, rhs)
        
        self.elements = elementwise(
            self.elements, rhs.elements, lambda a, b: a - b)
        
        return self
    
    def __rmul__(self, other):
        return self.__mul__(other)
        
    def __mul__(self, rhs):
        if isinstance(rhs, numbers.Number):
            return Matrix(scalar(rhs, self.elements, lambda a, b: a * b))
        
        if self.width != rhs.height:
            raise IncompatibleMatricesException(self, rhs)
        
        return Matrix(matrix_multiply(self.elements, rhs.elements))
    
    def __truediv__(self, rhs):
        return Matrix(scalar(rhs, self.elements, lambda a, b: a / b))
        
    def __eq__(self, rhs):
        flat = lambda x: itertools.chain(*x)
        return all([ math.isclose(a, b) for a, b in 
                        zip(flat(self.elements), flat(rhs.elements)) ])
    
    def __str__(self):
        return self.__repr__()
    
    def __repr__(self):
        return "\n".join([" ".join(str(e) for e in lst) 
                              for lst in self.elements])

    @staticmethod
    def from_lines(lines):
        tokens = map(lambda l: l.strip().split(), lines)
        elements = [ [float(n) for n in row] for row in tokens]
        return Matrix(elements)
        
    @staticmethod
    def from_file(location):
        with open(location) as f:
            return Matrix.from_lines(f)
    
    @staticmethod
    def to_file(matrix, location):
        with open(location) as f:
            f.write(matrix.__repr__())

In [46]:
Matrix.from_file('test.mat')

12.5 3.0 9.0 2.0
4.0 5.0 6.0 7.0
8.0 9.0 10.0 11.0

# Laboratorijske vjezbe

## 1.
Double varijable trebale bi se usporedjivati uz definiranu dozvoljenu gresku. U Pythonu 3.5 postoji methoda `math.isclose` koja radi takvu usporedbu ([referenca](https://docs.python.org/3/whatsnew/3.5.html#pep-485-a-function-for-testing-approximate-equality), default relativna tolerancija je `1e-09`). Tu metodu koristi klasa `Matrix` prilikom pozivanja `__eq__`.

In [47]:
m1 = Matrix([[123], [456]])
mul = 55

m2 = mul * m1
m3 = m2 / mul

In [48]:
m1 /= 2

In [49]:
m1

61.5
228.0

In [50]:
m1 = Matrix([[6.6]])
m2 = Matrix([[2.2 * 3.0]])

In [51]:
2.2 * 3.0 == 6.6

False

In [52]:
m1 == m2

True

## 2.

In [53]:
test_str = """2 3 1 5
6 13 5 19
2 19 10 23
4 10 11 31"""

In [54]:
book_example = Matrix.from_lines(test_str.split("\n"))

In [55]:
def assert_square_matrix(matrix):
    if matrix.width != matrix.height:
        raise InvalidDimensionException(matrix)

In [56]:
def assert_outside_epsilon(value, tolerance = 1e-9):
    if abs(value) <= tolerance:
        raise ValueError("Value below threshold tolerance ({0})" \
                            .format(tolerance))

In [57]:
def lower_triangular(matrix):
    assert_square_matrix(matrix)
    n = matrix.width
    
    lower_elements = []
    for row in range(n):
        row_vector = []
        for j in range(row):
            row_vector.append(matrix[row][j])
        
        row_vector.append(1)
        
        for j in range(row + 1, n):
            row_vector.append(0)
        lower_elements.append(row_vector)
    
    return Matrix(lower_elements)

In [58]:
def upper_triangular(matrix):
    assert_square_matrix(matrix)
    n = matrix.width
    
    upper_elements = []
    for row in range(n):
        row_vector = []
        for j in range(row):
            row_vector.append(0)
        for j in range(row, n):
            row_vector.append(matrix[row][j])
        
        upper_elements.append(row_vector)
        
    return Matrix(upper_elements)
    

In [59]:
def lu(matrix):
    assert_square_matrix(matrix)
        
    A = Matrix(matrix)
    n = A.width
    for i in range(n - 1):
        for j in range(i + 1, n):
            assert_outside_epsilon(A[i][i])
            A[j][i] /= A[i][i]
            for k in range(i + 1, n):
                A[j][k] = A[j][k] - (A[j][i] * A[i][k])
    L = lower_triangular(A)
    U = upper_triangular(A)
    return L, U

In [60]:
lu(book_example)

(1 0 0 0
 3.0 1 0 0
 1.0 4.0 1 0
 2.0 1.0 7.0 1, 2.0 3.0 1.0 5.0
 0 4.0 2.0 4.0
 0 0 1.0 2.0
 0 0 0 3.0)

In [61]:
def row_index_of_largest_number_in_column(A, P, col, n):
    pivot = col
        
    for j in range(col + 1, n):
        if abs(A[P[j]][col]) > abs(A[P[pivot]][col]):
            pivot = j
            
    return pivot

def permute_rows(A, P, n):
    P_inverse = dict(zip(range(n), P))
    return Matrix([ A[P_inverse[i]] for i in range(n) ])

def lup(A):
    assert_square_matrix(A)
    
    A = Matrix(A)
    n = A.width
    P = [i for i in range(n)]
    
    for i in range(n - 1):
        pivot = row_index_of_largest_number_in_column(A, P, i, n)        
        swap(P, i, pivot)
        
        assert_outside_epsilon(A[P[i]][i])
        for j in range(i + 1, n):
            A[P[j]][i] /= A[P[i]][i]
            for k in range(i + 1, n):
                A[P[j]][k] -= A[P[j]][i] * A[P[i]][k]
    
    result = permute_rows(A, P, n)
    return lower_triangular(result), upper_triangular(result), P

In [62]:
lup(book_example)

(1 0 0 0
 0.3333333333333333 1 0 0
 0.6666666666666666 0.09090909090909094 1 0
 0.3333333333333333 -0.09090909090909088 0.01315789473684211 1,
 6.0 13.0 5.0 19.0
 0 14.666666666666668 8.333333333333334 16.666666666666668
 0 0 6.909090909090909 16.81818181818182
 0 0 0 -0.039473684210526355,
 [1, 2, 3, 0])

In [63]:
def backward_sub(A, b):
    assert_square_matrix(A)
    n = b.width
    
    for i in reversed(range(n)):
        b[i] /= A[i][i]
        # premjesti coeff * x na desnu stranu (y' = y - c * x)
        for j in range(i - 1):
            b[j] = b[j] - A[j][i] * b[i]
            
    return b

def forward_sub(A, b):
    assert_square_matrix(A)
    n = b.width
    # premjesti coeff * x na desnu stranu (y' = y - c * x)
    for i in range(n - 1):
        for j in range(i + 1, n):
            b[j] = b[j] - A[j][i] * b[i]
            
    return b

In [64]:
m = Matrix(
    [[3, 9, 6],
     [4, 12, 12],
     [1, -1, 1]])

try:
    lu(m)
except ValueError as e:
    print("Encountered exception:", e.args[0])

Encountered exception: Value below threshold tolerance (1e-09)


## 3.

In [65]:
m = Matrix(
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]])
lu(m)

(1 0 0
 4.0 1 0
 7.0 2.0 1, 1 2 3
 0 -3.0 -6.0
 0 0 0.0)

In [66]:
lup(m)

(1 0 0
 0.14285714285714285 1 0
 0.5714285714285714 0.5000000000000002 1, 7 8 9
 0 0.8571428571428572 1.7142857142857144
 0 0 1.1102230246251565e-16, [2, 0, 1])

## 4.

In [67]:
lhs = Matrix(
    [[0.000001, 3000000, 2000000],
     [1000000, 2000000, 3000000],
     [2000000, 1000000, 2000000]])

rhs = Matrix(
    [[12000000.000001],
     [14000000],
     [10000000]])

In [68]:
L, U = lu(lhs)
a = forward_sub(L, rhs)
b = backward_sub(U, a)

In [69]:
b

12000000000001.0
14000000
10000000

In [70]:
L, U, P = lup(lhs)
lup(lhs)

(1 0 0
 5e-13 1 0
 0.5 0.5000000000000834 1, 2000000 1000000 2000000
 0 2999999.9999995 1999999.999999
 0 0 1000000.0000003333, [2, 0, 1])

In [73]:
rhs_prime = permute_rows(rhs, P, rhs.height)

In [74]:
a = forward_sub(L, rhs_prime.T)
b = backward_sub(U, a)
b

5.0
12000000000001.0
14000000

## 5.

In [39]:
lhs = Matrix(
    [[0, 1, 2],
     [2, 0, 3],
     [3, 5, 1]])

rhs = Matrix(
    [[6],
     [9],
     [3]])

## 6.

In [40]:
lhs = Matrix(
    [[4000000000, 1000000000, 3000000000],
     [4, 2, 7],
     [0.0000000003, 0.0000000005, 0.0000000002]])

rhs = Matrix(
    [[9000000000],
     [15],
     [0.0000000015]])


# TODO: Matlab
Potrebno je napisati skriptu za programski paket MATLAB koja će učitati matricu sustava i slobodni vektor
(iz istih datoteka kao programska implementacija), uz pomoć LUP dekompozicije pronaći i ispisati matrice
L, U i matricu permutacija P, te međurezultat y. Također naći rješenje sustava te invertiranu matricu sustava.
U skripti se (između ostalih) preporučuje koristiti sljedeće ugrađene funkcije: lu, inv te lijevo dijeljenje ( \ ).
Usporedite dobiveno rješenje sa onim koje ste dobili vašim programom. 